In [1]:
from instrumentdatabaseapi import instrumentdatabaseapi as API
repo = API.Repository(local_repo=".")

In [2]:
print(repo)

URL: https://github.com/PaNOSC-ViNYL/instrument_database.git
local_dir: ./


Load the instrument configuration providing:
 - name of the institute
 - name of the instrument
 - version of the instrument description (HEAD is the most recent)
 - name of the simulation program

In [3]:
#help(repo)

In [4]:
repo.ls_institutes()

 - EuXFEL
 - ILL
 - test_institute


In [5]:
repo.ls_instruments("ILL")
instrument_name="Panther"
#instrument_name="ThALES"
#instrument_name="IN5"
instrument_name="D11"

Available instruments for ILL:
 -  ILL/D22
 -  ILL/Panther
 -  ILL/ThALES
 -  ILL/IN5
 -  ILL/D11


In [6]:
repo.ls_versions("ILL",instrument_name)

Available versions for instrument D11:
 -  HEAD


In [7]:
repo.ls_simulation_programs("ILL",instrument_name,"HEAD")

Instrument D11 from institute ILL is implemented with the following programs:
 -  mcstas


In [8]:
repo.ls_flavours("ILL",instrument_name,"HEAD","mcstas")

Available flavours for instrument D11:
 -  None
 -  full
 -  nosection
 -  Borkron_1972
 -  Borkron_2003
 -  Borofloat_2001
 -  simple


In [9]:
flavour="full"
flavour="nosection"
#flavour="simple"
myinstrument = repo.load("ILL", instrument_name, "HEAD", "mcstas", flavour, dep=False)

0 (40.5 - collimation)>0 ? 1 : 0
1 (40.0 - collimation)>0 ? 6 : 0
2 (34.0 - collimation)>0 ? 6 : 0
3 (28.0 - collimation)>0 ? 1 : 0
4 (27.5 - collimation)>0 ? 7 : 0
5 (20.5 - collimation)>0 ? 4 : 0
6 (16.5 - collimation)>0 ? 3 : 0
7 (13.5 - collimation)>0 ? 3 : 0
8 (10.5 - collimation)>0 ? 2 : 0
9 (8.5 - collimation)>0 ? 1 : 0
10 (8.0 - collimation)>0 ? 2 : 0
11 (6.0 - collimation)>0 ? 1 : 0
12 (5.5 - collimation)>0 ? 1 : 0
13 (5.0 - collimation)>0 ? 1 : 0
14 (4.0 - collimation)>0 ? 1 : 0
15 (3.5 - collimation)>0 ? 1 : 0
16 (2.5 - collimation)>0 ? 1 : 0


In [10]:
myinstrument.list_calculators()

- Instrument: IN5 -
Calculators:
OriginCalc



In [11]:
print(myinstrument)

######## Instrument IN5
- Instrument: IN5 -
Calculators:
OriginCalc
- ParametersCollection object -

   OriginCalc
    - Parameters object -
   lambda                              6          [angstrom]              L[0.12 angstrom, 12 angstrom]
   dlambda                             0.1        [angstrom]              
   Vrpm                                4000                            velocity selector RPM   I[-inf dimensionless, 3100 dimensionless]I[28300, inf]I[9000, 11900]
   collimation                         1.5        [meter]              Collimation length: free path between end of the guide and sample   (40.5 meter, 34 meter, 28 meter, 20.5 meter, 16.5 meter, 13.5 meter, 10.5 meter, 8 meter, 5.5 meter, 4 meter, 2.5 meter, 1.5 meter)
   sample_y_rotation                   0          [degree]             sample table rotation angle   
   sample_width                        0.02       [meter]              width of the sample if shape is box   
   sample_height                 

In [12]:
# setting the base directory for the simulation output
basedir = "/tmp/"+instrument_name+"/"
myinstrument.set_instrument_base_dir(basedir)
import os 
#os.mkdir(basedir)


In [13]:
myinstrument.master

 - Parameters object -

In [14]:
import pint
ureg = pint.get_application_registry()

if instrument_name == "ThALES":
    myinstrument.master["a2"] = myinstrument.energy_to_angle(4.98 * ureg.meV)
    myinstrument.master["a4"] = 60 * ureg.degree
    myinstrument.master["a6"] = myinstrument.master["a2"].pint_value
if instrument_name == "Panther":
    myinstrument.master["energy"]=19 * ureg.meV

In [15]:
myinstrument.samples

['None', 'vanadium', 'H2O', 'D2O', 'sqw']

In [16]:
myinstrument.set_sample_by_name("sqw")
myinstrument.master["sqw_file"]='"/users/nourbakhsh/digitaltwin/instrument_database/institutes/ILL/instruments/D11/HEAD/mcstas/data/simul_5711.dat"'

In [17]:
myinstrument.sample_holder(material="quartz", shape="box",w=0.02, h=0.03, d=0.0135, th=0.00125)

COMPONENT sample_holder_in = Isotropic_Sqw(
  Sqw_coh = "SiO2_quartza.laz", // [str]
  Sqw_inc = 0, // [str]
  radius = sample_holder_radius, // [m]
  thickness = sample_holder_thickness, // [m]
  xwidth = sample_holder_width, // [m]
  yheight = sample_holder_height, // [m]
  zdepth = sample_holder_depth, // [m]
  verbose = 2, // [1]
  concentric = 1, // [1]
  p_interact = 1 // [1]
)
AT (0, 0, 0) RELATIVE sample_arm
COMPONENT sample_holder_out = Isotropic_Sqw(
  Sqw_coh = "SiO2_quartza.laz", // [str]
  Sqw_inc = 0, // [str]
  radius = sample_holder_radius, // [m]
  thickness = sample_holder_thickness, // [m]
  xwidth = sample_holder_width, // [m]
  yheight = sample_holder_height, // [m]
  zdepth = sample_holder_depth, // [m]
  verbose = 2, // [1]
  concentric = 0, // [1]
  p_interact = 1 // [1]
)
AT (0, 0, 0) RELATIVE sample_arm


In [18]:
#myinstrument.sample_box_shape(0.02, 0.03,0.0035-2*0.00125)
myinstrument.sample_shape("holder")

In [19]:
myinstrument.calculators["OriginCalc"].parameters

 - Parameters object -
lambda                              6          [angstrom]              L[0.12 angstrom, 12 angstrom]
dlambda                             0.1        [angstrom]              
Vrpm                                4000                            velocity selector RPM   I[-inf dimensionless, 3100 dimensionless]I[28300, inf]I[9000, 11900]
collimation                         1.5        [meter]              Collimation length: free path between end of the guide and sample   (40.5 meter, 34 meter, 28 meter, 20.5 meter, 16.5 meter, 13.5 meter, 10.5 meter, 8 meter, 5.5 meter, 4 meter, 2.5 meter, 1.5 meter)
sample_y_rotation                   0          [degree]             sample table rotation angle   
sample_width                        0.02       [meter]              width of the sample if shape is box   
sample_height                       0.03       [meter]              height of the sample if shape is box or cylinder   
sample_depth                        0.011      [m

In [20]:
myinstrument.sim_neutrons(5e7)
myinstrument.set_seed(654321)
myinstrument.custom_flags("-I mcstas/components")

In [21]:
calc =myinstrument.calculators["OriginCalc"]
print(calc.parameters)
calc.parameters["detpos"]=28

 - Parameters object -
lambda                              6          [angstrom]              L[0.12 angstrom, 12 angstrom]
dlambda                             0.1        [angstrom]              
Vrpm                                4000                            velocity selector RPM   I[-inf dimensionless, 3100 dimensionless]I[28300, inf]I[9000, 11900]
collimation                         1.5        [meter]              Collimation length: free path between end of the guide and sample   (40.5 meter, 34 meter, 28 meter, 20.5 meter, 16.5 meter, 13.5 meter, 10.5 meter, 8 meter, 5.5 meter, 4 meter, 2.5 meter, 1.5 meter)
sample_y_rotation                   0          [degree]             sample table rotation angle   
sample_width                        0.02       [meter]              width of the sample if shape is box   
sample_height                       0.03       [meter]              height of the sample if shape is box or cylinder   
sample_depth                        0.011      [m

In [22]:
myinstrument.run()

---- Found 1 places in McStas output with keyword 'error'. 

ERROR          (sigma_coh, sigma_inc).
Isotropic_Sqw: sqw: Focusing can only apply for single
               scattering. Setting to order=1.
Detector: sqw_coh.sq_I=1504.54 sqw_coh.sq_ERR=2.90732 sqw_coh.sq_N=518 "sqw_coh.sq"
Detector: sqw_coh.sigma_I=494.837 sqw_coh.sigma_ERR=0.956208 sqw_coh.sigma_N=518 "sqw_coh.sigma"
Isotropic_Sqw: sample_holder_out: is a hollow box: width=0.020000 height=0.030000 depth=0.013500 
Opening input file '/usr/share/mcstas/3.3/tools/Python/mcrun/../mccodelib/../../../data/SiO2_quartza.laz' (Table_Read_Offset)
Opening input file '/usr/share/mcstas/3.3/tools/Python/mcrun/../mccodelib/../../../data/SiO2_quartza.laz' (Table_Read_Offset)
Isotropic_sqw: Converting 625 powder lines from SiO2_quartza.laz into S(q,w) data
Isotropic_sqw: q range [0:12.4939], creating 25254 elements vector
----------------------------------------------------------------------

loading system configuration
INFO: Using direc

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xef in position 3037: invalid continuation byte

### Diagnostics of the beam for one calculator

In [ ]:
calculator_name="SampleCalc"
calculator_name="OriginCalc"
mycalc = myinstrument.calculators[calculator_name]
#mycalc.backengine()
#myinstrument.run()
#print(mycalc.output)

In [ ]:
#mycalc = myinstrument.calculators["OriginCalc"]
#mycalc = myinstrument.calculators["SampleCalc"]
#mycalc.input = myinstrument.calculators["OriginCalc"].output
#mycalc.show_diagram()

"sqw_file" in mycalc.parameters


In [ ]:
mycalc.show_diagram(analysis=True, variable=None)

In [ ]:
import mcstasscript as ms


In [ ]:
diag = ms.Diagnostics(mycalc)
diag.settings(ncount=1e5, suppress_output=True)
diag.show_settings()
diag.clear_points()


for component in diag.instr.component_list:
        if component.component_name in ["Progress_bar", "Arm"]:
            continue
        if component.component_name in ["Beamstop"]:
            diag.add_point(before=component.name)
            continue
        if component.name in ["VCS", "HCS"]:
            diag.add_point(after=component.name)
            continue
        if component.name in ["Chopper0","Chopper1","Chopper2","Chopper3"]:
            diag.add_point(before=component.name)
            diag.add_point(after=component.name)
        else:
            #print("#"+component.name+"#")
            diag.add_point(before=component.name)
        



if mycalc.name == "SampleCalc":
    diag.clear_points()
    diag.add_point(after="Vin")
    diag.add_point(before="sqw")
    diag.add_point(after="Sample_Out")
    diag.add_point(before="after_sample_slit")
    diag.add_point(after="after_sample_slit")

#print(diag)


In [ ]:
#mycalc.get_component("BC1").phase=41
#ch = mycalc.get_component("Chopper3")
#print(ch)
diag.settings(ncount=1e6)

In [ ]:
diag.run()

In [ ]:
diag.clear_views()
#diag.add_view("t")
#diag.add_view("x", bins=50, limits=[-0.04,0.04])
#diag.add_view("y", bins=50, same_scale=False, limits=[-0.07,0.07])
#diag.add_view("x","y",bins=[30,30])
#diag.add_view("e", same_scale=True)
diag.add_view("l",same_scale=False)
diag.add_view("t",same_scale=False)
#diag.add_view("dx","dy")
diag.add_view("t","x",same_scale=False)
diag.add_view("t","y", same_scale=False)
diag.add_view("x","y",same_scale=False)
diag.add_view("vz",same_scale=False,left_lim=700)
print(diag)

In [ ]:
diag.plot()

In [ ]:
%matplotlib widget
import mcstasscript.jb_interface as ms_widget
#ms_widget.show(data_empty)

In [ ]:
#myinstrument.sample.set_SPLIT(60)
#myinstrument.sample

In [ ]:
#slit=myinstrument.calculators["SampleCalc"].get_component("after_sample_slit")
#slit.EXTEND=""
#slit.append_EXTEND("if(p<1e-1) ABSORB;")
#slit


In [ ]:
#myinstrument.run()

In [ ]:
data=myinstrument.output
#print(data)
calcname_data="DetectorCalc_data"
calcname_data="OriginCalc_data"
detectors=data[calcname_data].get_data()["data"]

In [ ]:
#print(data[calcname_data])

#print()
for detector in detectors:
    print(detector, "\n\t I = ",detector.Intensity, "\n\t E =", detector.Error, "\n\t N =", detector.Ncount)

In [ ]:
print(detectors)
#help(detector)
import mcstasscript as ms
ms.make_sub_plot(detectors[-3:])

In [ ]:
detector = detectors[3]
print(detector.Intensity)
import numpy as np
help(detector.Intensity)

In [ ]:
wmins=["1e-5", "1e-4", "1e-3", "1e-2", "1e-1"]
detectors=[]
for w_min in wmins:
    slit=myinstrument.calculators["SampleCalc"].get_component("after_sample_slit")
    slit.EXTEND=""
    slit.append_EXTEND(f"if(p<{w_min}) ABSORB;")
    myinstrument.run()
    data=myinstrument.output
    detector=(data["DetectorCalc_data"].get_data()["data"][0])
    detectors.append(detector)

In [ ]:
detectors

In [ ]:
import numpy as np
diags = []
RHs = list(np.arange(2.6,4,0.3))
RVs = list(np.arange(1,3,0.3))
RVs.append(1.88)
RHs.append(3.31)
print(RHs)
print(RVs)

In [ ]:
for RH in RHs:
    print(RH)
    mycalc.parameters["RHmono"]=RH
    diag = ms.Diagnostics(mycalc)
    diag.settings(ncount=1e6, suppress_output=True,force_compile=False)
    diag.clear_points()
    diag.add_point(after="before_sample_slit")
#    diag.add_point(before="sqw")
    diag.run()
    diags.append(diag)

#(RHmono,RVmono) = (3.31,1.88)
#lambda: 4.05
#lambda: 4.05 +/- 0.04
#energy: 4.98 +/- 0.10
#(RHanalyzer,RVanalyzer) = (4.17,0.77)
#'Monochromator', 'counter', 'before_sample_slit', 'sqw', 'after_sample_slit', 'analyzer', 'slit', 'detector_all']

In [ ]:
for diag in diags:
    diag.clear_views()
    diag.add_view("x", bins=50, limits=[-0.04,0.04])
    diag.add_view("y", bins=50, same_scale=False, limits=[-0.07,0.07])
    #diag.add_view("x","y",bins=[30,30])
    #diag.add_view("dx","dy")
    diag.add_view("dx")
    diag.add_view("dy")
    diag.plot()

In [ ]:
RVdiags = []


print(RVs)
for RV in RVs:
    print(RV)
    mycalc.parameters["RVmono"]=RV
    diag = ms.Diagnostics(mycalc)
    diag.settings(ncount=1e6, suppress_output=True,force_compile=False)
    diag.clear_points()
    diag.add_point(after="before_sample_slit")
#    diag.add_point(before="sqw")
    diag.run()
    RVdiags.append(diag)

In [ ]:
for diag in RVdiags:
    diag.clear_views()
    diag.add_view("x", bins=50, limits=[-0.04,0.04])
    diag.add_view("y", bins=50, same_scale=False, limits=[-0.07,0.07])
    #diag.add_view("x","y",bins=[30,30])
    #diag.add_view("dx","dy")
    diag.add_view("dx")
    diag.add_view("dy")
    diag.plot()

In [ ]:
myinstrument.force_compile(False)
myinstrument.sim_neutrons(1e5)
counter=[]
detector=[]
mycalc = myinstrument.calculators["OriginCalc"]
for RV in RVs:
    for RH in RHs:
        mycalc.parameters["RHmono"].value = RH
        mycalc.parameters["RVmono"].value = RV
        #print(mycalc.parameters)
        myinstrument.run()
        counter.append(myinstrument.output["OriginCalc_data"].get_data()["data"])

In [ ]:
i=0
i_det=1
for RV in RVs:
    for RH in RHs:
        c = counter[i]
        det=c[1]
        print(i, "\t" , RV, RH, "\t\t" , det.Intensity, det.Error, det.Ncount)
        i=i+1

In [ ]:
mycalc.output

In [ ]:
print(myinstrument.output)

In [ ]:
RHs = list(np.arange(1.5,3,0.2))
RVs = list(np.arange(0.3,1.2,0.2))
RVs.append(0.51)
RHs.append(1.41)
print(RHs)
print(RVs)

myinstrument.force_compile(False)
myinstrument.sim_neutrons(1e5)
counter=[]

mycalc = myinstrument.calculators["OriginCalc"]
for RV in RVs:
    for RH in RHs:
        mycalc.parameters["RHanalyzer"].value = RH
        mycalc.parameters["RVanalyzer"].value = RV
        #print(mycalc.parameters)
        myinstrument.run()
        counter.append(myinstrument.output["OriginCalc_data"].get_data()["data"])

In [ ]:
i=0
i_det=1
intensities = []
counts = []
for RV in RVs:
    for RH in RHs:
        c = counter[i]
        det=c[1]
        #print(i, "\t" , RV, RH, "\t\t" , det.Intensity, det.Error, det.Ncount)
        intensities.append(det.Intensity)
        counts.append(det.Ncount)
        i=i+1

In [ ]:
max(intensities)

In [ ]:
max(counts)

In [ ]:
intensities.index(max(intensities))

In [ ]:
counts.index(max(counts))

In [ ]:
i=0
for RV in RVs:
    for RH in RHs:
        print(RV, RH, counts[i])
        if i==counts.index(max(counts)):
            print("---> ",RV, RH)
        i=i+1

In [ ]:
RVs

In [ ]:
RHs

In [ ]:
dist_sample_ana = 1.260
dist_ana_det = 0.640
import math
ana_focus = 1.0 / (1.0 / dist_sample_ana + 1.0 / dist_ana_det)

In [ ]:

def rh(ana_focus):
    return 2*ana_focus/math.sin(math.pi/180*myinstrument.master["a6"].value/2)
def rv(ana_focus):
    return 2*ana_focus*math.sin(math.pi/180*myinstrument.master["a6"].value/2)


In [ ]:
print(rv(ana_focus),rh(ana_focus))

In [ ]:
ana_focus=dist_ana_det
print(rv(ana_focus),rh(ana_focus))